This python script reads file attachments from emails, converts into JSON after removing unwanted fields and required formatting, and then stores the JSON in Google Cloud Storage

In [1]:
#Import the required libraries
import email
import getpass
import imaplib
import os
import sys
import numpy as np
import pandas as pd
import json
import pandas.io.gbq as gbq
from google.cloud import bigquery

In [6]:
def readEmailAttachment(from_user):
    #IMAP Email User/Password
    userName = 'codebalanceit@gmail.com'
    password = 'Hari@coding'
    
    imapSession = imaplib.IMAP4_SSL('imap.google.com')
    typ, accountDetails = imapSession.login(user, pwd)
    
    imapSession.select()
    typ, data = imapSession.search(None, '(FROM "'+from_user+'") UNSEEN')
    
    #iterate through all emails
    fileNames=[]
    for msgId in data[0].split():
        typ, messageParts = imapSession.fetch(mgsId, '(RFC822)')
        #get email body
        emailBody = messageParts[0][1]
        mail = email.message_from_bytes(emailBody)
        for part in mail.walk():
            fileName.append(part.get_filename())
            
    return fileName

In [7]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    # upload a file to bucket
    # Initiate a client
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    
    blob.upload_from_filename(source_file_name)
    
    print('File {} uploaded to {}.'.format(source_file_name,
        destination_blob_name))

In [ ]:
def readLoadTracking():
    client = bigquery.Client()
    query = """
        select * from sarasmaster-201918:kopari.load_tracking
        WHERE data_source = @source
        AND date_from = @date_from
        AND date_to = @date_to;
        """
    query_params = [
        bigquery.ScalarQueryParameter('source', 'STRING', source)
        bigquery.ScalarQueryParameter('date_from', 'STRING', date_from)
        bigquery.ScalarQueryParameter('date_to', 'STRING', date_to)
    ]
    job_config = bigquery.QueryJobConfig()
    job_config.query_parameters = query_params
    query_job = client.query(query, job_config=job_config)

    query_job.result()  # Wait for job to complete

    # Print the results.
    destination_table_ref = query_job.destination
    table = client.get_table(destination_table_ref)
    table_data = None
    for row in client.list_rows(table):
        if row:
            return('Record exists')
        else:
            return True   

In [ ]:
def updateLoadTracking(table, rows):
    errors = client.insert_rows(table, rows)  # API request
    assert errors == []

In [3]:
# Get all file attachments
fileNames = readEmailAttachment("Sreehari Nambiar")

In [ ]:
if not fileNames:
    for fileName in fileNames:
        if 'Ulta' in fileName:
            upload_blob('sarasmaster', fileName, 'kopari/retail/ulta/ulta_sales.xls')
        elif 'Nordstorm' in fileName:
            upload_blob('sarasmaster', fileName, 'kopari/retail/ulta/nordstrom_sales.xls')
        elif 'Sephora' in fileName:
            upload_blob('sarasmaster', fileName, 'kopari/retail/ulta/sephora_sales.xls')
            